Getting rootpath of this project

In [1]:
import os
root = os.path.dirname(os.path.dirname(os.getcwd()))

Loading fixed data

In [2]:
import pandas as pd

data = pd.read_csv(os.path.join(root, "data", "processed", "cleanData.csv"))
data.head()

,index,age,sex,trestbps,chol,thalach,oldpeak,trf,fbs_0,fbs_1,...,hc_0,hc_1,hc_2,sk_0,sk_1,sk_2,sk_3,slope_0.0,slope_1.0,slope_2.0
0,0,49,M,130,269.0,163,0.0,113,1,0,...,1,0,0,0,1,0,0,0,0,1
1,1,61,F,138,166.0,125,3.6,71,1,0,...,0,1,0,0,0,0,1,1,0,0
2,2,46,F,140,311.0,120,1.8,68,1,0,...,1,0,0,0,1,0,0,1,0,0
3,3,69,F,140,254.0,146,2.0,119,1,0,...,0,1,0,1,0,0,0,0,1,0
4,4,51,F,100,222.0,143,1.2,92,1,0,...,0,1,0,1,0,0,0,0,1,0


## split data into target (Y) and paramters (X)

In [3]:
## as I said in the last notebook, I will not use age
x, y = data.iloc[:,3:], data.iloc[:,2]

In [4]:
x.head()

,trestbps,chol,thalach,oldpeak,trf,fbs_0,fbs_1,restecg_0,restecg_1,restecg_2,...,hc_0,hc_1,hc_2,sk_0,sk_1,sk_2,sk_3,slope_0.0,slope_1.0,slope_2.0
0,130,269.0,163,0.0,113,1,0,0,1,0,...,1,0,0,0,1,0,0,0,0,1
1,138,166.0,125,3.6,71,1,0,1,0,0,...,0,1,0,0,0,0,1,1,0,0
2,140,311.0,120,1.8,68,1,0,0,1,0,...,1,0,0,0,1,0,0,1,0,0
3,140,254.0,146,2.0,119,1,0,1,0,0,...,0,1,0,1,0,0,0,0,1,0
4,100,222.0,143,1.2,92,1,0,0,1,0,...,0,1,0,1,0,0,0,0,1,0


In [5]:
y.head()

0    M
1    F
2    F
3    F
4    F
Name: sex, dtype: object

Since it's a small dataset, and I will not use deep learning, I gonna use sklern and xgboost to train some models.

In [6]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression(
    penalty='l2',
    C=0.1,
    solver = 'liblinear'
)

sc = cross_val_score(logistic_model, x, y, cv=8)

print(" Logistic Regression \n Mean:{} \n Min:{} \n Max:{}".format(sc.mean(), sc.min(), sc.max()))

 Logistic Regression 
 Mean:0.7704014939309057 
 Min:0.6857142857142857 
 Max:0.8235294117647058


In [7]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    max_depth=2,
    gamma=3,
    eta=0.8,
    reg_alpha=0.5,
    reg_lambda=0.5
)

sc = cross_val_score(xgb_model, x, y, cv=8)
print(" XGB \n Mean:{} \n Min:{} \n Max:{}".format(sc.mean(), sc.min(), sc.max()))

 XGB 
 Mean:0.7850840336134454 
 Min:0.6944444444444444 
 Max:0.9142857142857143


In [8]:
from sklearn.ensemble import RandomForestClassifier

randomF_model = RandomForestClassifier(n_estimators = 50, max_depth=4)
sc = cross_val_score(randomF_model, x, y, cv=8)
print(" Random Forest \n Mean:{} \n Min:{} \n Max:{}".format(sc.mean(), sc.min(), sc.max()))

 Random Forest 
 Mean:0.7531395891690009 
 Min:0.6666666666666666 
 Max:0.8235294117647058


After we see that they all performe allmost the same, just a little beyond the basemodel (69%), I gonna try to ensemble the models and see if if the enseble permorms better.

In [9]:
from sklearn.ensemble import VotingClassifier
 
ens = VotingClassifier(estimators=[
    ('logisticRegression', logistic_model),
    ('XGB', xgb_model),
    ('randomForest', randomF_model)], voting='soft')

scor = cross_val_score(ens, x, y, cv=8)
print(" Ensemble \n Mean:{} \n Min:{} \n Max:{}".format(scor.mean(), scor.min(), scor.max()))

 Ensemble 
 Mean:0.7639589169000933 
 Min:0.6666666666666666 
 Max:0.8529411764705882


I was hoping get better results than individual ones, it was not the case, so I will try to just use two

In [10]:
estimatores_test = [[('logisticRegression', logistic_model), ('randomForest', randomF_model)],
                   [('logisticRegression', logistic_model), ('XGB', xgb_model)],
                   [('XGB', xgb_model), ('randomForest', randomF_model)]]

for estimat in estimatores_test:
    ens = VotingClassifier(estimators=estimat, voting='soft')
    scor = cross_val_score(ens, x, y, cv=8)
    print(" Ensemble ({}) \n Mean:{} \n Min:{} \n Max:{}".format(estimat, scor.mean(), scor.min(), scor.max()))


 Ensemble ([('logisticRegression', LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)), ('randomForest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=4, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False))]) 
 Mean:0.7600898692810458 
 Min:0.6666666666666666 
 Max:0.8529411764705882
 Ensemble ([('logisticRegression', LogisticRegress

## Conclusion
Well, since I couldn't get an better result than an single model I will stick wit XGB.
I could do an grid search for better hyperparamters, but I already tried most of the ones that I belive that would improve the model.

First I gonna train the model with all the paramters and after that I will save

In [11]:
import pickle

final_xgb = xgb_model.fit(x, y)

# save model to file
pickle.dump(final_xgb, open(os.path.join(root, "models", "model.pickle.dat"), "wb"))